<a href="https://colab.research.google.com/github/ykitaguchi77/YOLACT_Mobius/blob/main/Mobius2label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Mobius2label**

Mobiusのカラーマスク→ YOLOv8 instance trainingトレーニング用の座標に

研究室配属 鈴木くんより

In [1]:
#@title #ライブラリのインストール
#@markdown [shift]+[Enter]を押して起動してください
from google.colab import drive
drive.mount('/content/drive')
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
import json
from tqdm import tqdm  # tqdmをインポート
import os
import matplotlib

Mounted at /content/drive


In [ ]:
#@title #ディレクトリと座標の個数
#@markdown 指定を終えてから、[shift]+[Enter]を押して起動してください。
#@markdown <br> 学習に適切な個数が分からなかったので、変更できるようにしました。

# 入力ディレクトリと出力ディレクトリを指定してください
input_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks/1"
output_dir = "/content"

# eyelid, iris, pupilごとに座標の個数を指定してください。
num_points_eyelid = 24
num_points_iris = 16
num_points_pupil = 12

In [ ]:
#@title #Mobius to label起動
#@markdown 　[shift]+[Enter]を押して起動してください。
#@markdown　<br>うまくいっていないラベルも存在するので、目視で確認して除外すべきかもしれません。
def create_mask(image, color):
    return np.all(image == color, axis=-1)

def apply_mask(image, mask):
    return np.where(mask, 255, 0)

def extract_contours(img, _num_points):
    contours = measure.find_contours(img, 0.5)
    points_list = []

    for contour in contours:
        contour_length = len(contour)

        if contour_length > _num_points:
            step = contour_length // _num_points
            selected_points = [contour[i * step] for i in range(_num_points)]
        else:
            selected_points = contour

        points = [[float(point[1]), float(point[0])] for point in selected_points]
        points_list.append(points)

    return points_list

def create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file):
    with open(output_file, 'w') as f:
        # 0: eyelid
        f.write("0 ")
        for point in points_list_all_colors[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

        # 1: iris
        f.write("1 ")
        for point in points_list_blue_green[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

        # 2: pupil
        f.write("2 ")
        for point in points_list_blue[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x} {y} ")
        f.write("\n")

def process_image(mask_file, output_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    img = cv2.imread(mask_file, cv2.IMREAD_COLOR)
    img_height, img_width = img.shape[:2]

    mask_blue = create_mask(img, [255, 0, 0])
    mask_green = create_mask(img, [0, 255, 0])
    mask_red = create_mask(img, [0, 0, 255])

    mask_blue_green = np.logical_or(mask_blue, mask_green)
    mask_all_colors = np.logical_or(mask_blue_green, mask_red)

    img_blue = apply_mask(img, mask_blue)
    img_blue_green = apply_mask(img, mask_blue_green)
    img_all_colors = apply_mask(img, mask_all_colors)

    # 各ラベルごとに異なる個数の座標を抽出
    points_list_blue = extract_contours(img_blue, num_points_pupil)
    points_list_blue_green = extract_contours(img_blue_green, num_points_iris)
    points_list_all_colors = extract_contours(img_all_colors, num_points_eyelid)

    # 出力先ディレクトリとファイル名を設定
    base_name = os.path.splitext(os.path.basename(mask_file))[0]
    output_file = os.path.join(output_dir, base_name + ".txt")

    # カスタム出力
    create_custom_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file)

    # 書き換え部分
    x_eyelid = [point[0] for point in points_list_all_colors[0]]
    y_eyelid = [point[1] for point in points_list_all_colors[0]]

    x_iris = [point[0] for point in points_list_blue_green[0]]
    y_iris = [point[1] for point in points_list_blue_green[0]]

    x_pupil = [point[0] for point in points_list_blue[0]]
    y_pupil = [point[1] for point in points_list_blue[0]]

    # 元の画像を読み込む
    img_path = mask_file.replace("Masks", "Images").replace(".png", ".jpg")
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    # プロット作成
    plt.figure(figsize=(12, 6))

    # 左側に元の画像を表示
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(base_name)  # ファイル名をタイトルに設定

    # 右側に散布図を表示
    plt.subplot(1, 2, 2)
    plt.scatter(x_eyelid, y_eyelid, label='eyelid', color='blue')
    plt.scatter(x_iris, y_iris, label='iris', color='green')
    plt.scatter(x_pupil, y_pupil, label='pupil', color='red')
    plt.title("Contours")  # 輪郭

    # 軸を画像の大きさに合わせて調整
    plt.xlim([0, img_width])
    plt.ylim([img_height, 0])  # Y軸の反転

    # 縦横の比率を保つ
    plt.gca().set_aspect('equal', adjustable='box')

    # 凡例を表示する
    plt.legend()

    # レイアウトを調整して表示
    plt.tight_layout()
    plt.show()





def process_all_images_in_directory(input_dir, output_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    # 入力ディレクトリ内のすべての.pngファイルを処理
    png_files = [file_name for file_name in os.listdir(input_dir) if file_name.endswith(".png")]

    # tqdmを使って進行状況を表示
    for file_name in tqdm(png_files, desc="Processing images"):
        mask_file = os.path.join(input_dir, file_name)
        process_image(mask_file, output_dir, num_points_eyelid, num_points_iris, num_points_pupil)



if __name__ == '__main__':
    process_all_images_in_directory(input_dir, output_dir, num_points_eyelid, num_points_iris, num_points_pupil)


###一つだけの画像バージョン（テスト用）

# 備考欄

## ラベルの構造

0 x1 y1 x2 y2 x3 y3...(eyelidの座標)<br>
1 x1 y1 x2 y2 x3 y3...(irisの座標)<br>
2 x1 y1 x2 y2 x3 y3...(pupilの座標)

#**YOLOv8 instance segmentation training**

To do：

Mobiusのマスク画像をYOLOv8形式のテキストファイルに

マスク画像がない画像を除外

Not goodの画像を除外

YOLOv8用のフォルダ構成に

Yolov8のtraining

マスク画像がない画像をinference

手動で選別しOKのものを追加し再学習

In [ ]:
"""
project/
│
├── data/
│   ├── train/
│   │   ├── images/
│   │   └── labels/
│   │
│   ├── val/
│   │   ├── images/
│   │   └── labels/
│   │
│   └── test/  (オプション)
│       ├── images/
│       └── labels/
│
│
├── dataset.yaml
│
├── models/
│
└── runs/  (トレーニング結果が保存される)
"""

###Mask2label preprocessing



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
このスクリプトは、Mobiusデータセットの画像とマスクをYOLOv8形式に変換します。

主な機能：
1. Masksディレクトリ内の全てのマスク画像を処理（フォルダ番号順）
2. 各マスクに対応するYOLOv8形式のラベルファイルを生成
3. 対応する前眼部画像を指定された出力ディレクトリにコピー
4. 処理中に画像とマスクのプロットを表示
5. tqdmを使用して処理の進捗を表示
6. 開始フォルダを指定可能

入力ディレクトリ構造：
- Masks: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks/
  - 1/
  - 2/
  ...
  - 35/
- Images: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Images/
  - 1/
  - 2/
  ...
  - 35/

出力ディレクトリ構造：
- Labels: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/labels/
- Images: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/images/

注意：
- 出力ディレクトリが存在しない場合は自動的に作成されます
- マスクの色に基づいて、瞼（赤）、虹彩（緑）、瞳孔（青）を識別します
- 各部位の座標点の数は変数で設定可能です（デフォルト：瞼24点、虹彩16点、瞳孔12点）
- フォルダは番号順に処理されます
- 開始フォルダ番号を指定できます
"""

import os
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

def create_mask(image, color):
    """
    指定された色に基づいてマスクを作成します。

    Args:
    image (np.array): 入力画像
    color (list): マスクを作成する色 [R, G, B]

    Returns:
    np.array: 指定された色に対するバイナリマスク
    """
    return np.all(image == color, axis=-1)

def apply_mask(image, mask):
    """
    画像にマスクを適用します。

    Args:
    image (np.array): 入力画像
    mask (np.array): バイナリマスク

    Returns:
    np.array: マスクが適用された画像
    """
    return np.where(mask, 255, 0)

def extract_contours(img, _num_points):
    """
    画像から輪郭を抽出し、指定された数の点を選択します。

    Args:
    img (np.array): 入力画像
    _num_points (int): 選択する点の数

    Returns:
    list: 選択された点のリスト
    """
    contours = measure.find_contours(img, 0.5)
    points_list = []

    for contour in contours:
        contour_length = len(contour)

        if contour_length > _num_points:
            step = contour_length // _num_points
            selected_points = [contour[i * step] for i in range(_num_points)]
        else:
            selected_points = contour

        points = [[float(point[1]), float(point[0])] for point in selected_points]
        points_list.append(points)

    return points_list

def create_yolov8_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file):
    """
    YOLOv8形式のラベルファイルを作成します。

    Args:
    points_list_blue (list): 瞳孔の点のリスト
    points_list_blue_green (list): 虹彩の点のリスト
    points_list_all_colors (list): 瞼の点のリスト
    img_width (int): 画像の幅
    img_height (int): 画像の高さ
    output_file (str): 出力ファイルのパス
    """
    with open(output_file, 'w') as f:
        # 0: eyelid
        f.write("0 ")
        for point in points_list_all_colors[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x:.6f} {y:.6f} ")
        f.write("\n")

        # 1: iris
        f.write("1 ")
        for point in points_list_blue_green[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x:.6f} {y:.6f} ")
        f.write("\n")

        # 2: pupil
        f.write("2 ")
        for point in points_list_blue[0]:
            x = point[0] / img_width
            y = point[1] / img_height
            f.write(f"{x:.6f} {y:.6f} ")
        f.write("\n")

def process_image(mask_file, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil):
    """
    1つのマスク画像を処理し、対応するラベルファイルを生成し、元の画像をコピーします。

    Args:
    mask_file (str): マスク画像のファイルパス
    images_dir (str): 元の画像が格納されているディレクトリ
    output_labels_dir (str): ラベルファイルの出力ディレクトリ
    output_images_dir (str): 画像のコピー先ディレクトリ
    num_points_eyelid (int): 瞼の点の数
    num_points_iris (int): 虹彩の点の数
    num_points_pupil (int): 瞳孔の点の数
    """
    img = cv2.imread(mask_file, cv2.IMREAD_COLOR)
    img_height, img_width = img.shape[:2]

    mask_blue = create_mask(img, [255, 0, 0])
    mask_green = create_mask(img, [0, 255, 0])
    mask_red = create_mask(img, [0, 0, 255])

    mask_blue_green = np.logical_or(mask_blue, mask_green)
    mask_all_colors = np.logical_or(mask_blue_green, mask_red)

    img_blue = apply_mask(img, mask_blue)
    img_blue_green = apply_mask(img, mask_blue_green)
    img_all_colors = apply_mask(img, mask_all_colors)

    points_list_blue = extract_contours(img_blue, num_points_pupil)
    points_list_blue_green = extract_contours(img_blue_green, num_points_iris)
    points_list_all_colors = extract_contours(img_all_colors, num_points_eyelid)

    base_name = os.path.splitext(os.path.basename(mask_file))[0]
    output_file = os.path.join(output_labels_dir, base_name + ".txt")

    create_yolov8_output(points_list_blue, points_list_blue_green, points_list_all_colors, img_width, img_height, output_file)

    # Copy corresponding image
    folder_number = os.path.basename(os.path.dirname(mask_file))
    image_path = os.path.join(images_dir, folder_number, base_name + ".jpg")
    if os.path.exists(image_path):
        shutil.copy(image_path, output_images_dir)
    else:
        print(f"Warning: Corresponding image not found for {base_name}")

    # # Plot
    # x_eyelid = [point[0] for point in points_list_all_colors[0]]
    # y_eyelid = [point[1] for point in points_list_all_colors[0]]
    # x_iris = [point[0] for point in points_list_blue_green[0]]
    # y_iris = [point[1] for point in points_list_blue_green[0]]
    # x_pupil = [point[0] for point in points_list_blue[0]]
    # y_pupil = [point[1] for point in points_list_blue[0]]

    # img_original = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 2, 1)
    # plt.imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))
    # plt.title(base_name)

    # plt.subplot(1, 2, 2)
    # plt.scatter(x_eyelid, y_eyelid, label='eyelid', color='blue')
    # plt.scatter(x_iris, y_iris, label='iris', color='green')
    # plt.scatter(x_pupil, y_pupil, label='pupil', color='red')
    # plt.title("Contours")
    # plt.xlim([0, img_width])
    # plt.ylim([img_height, 0])
    # plt.gca().set_aspect('equal', adjustable='box')
    # plt.legend()
    # plt.tight_layout()
    # plt.show()

def process_all_images_in_directory(masks_dir, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil, start_folder):
    """
    指定されたディレクトリ内の全ての画像を処理します。

    Args:
    masks_dir (str): マスク画像が格納されているディレクトリ
    images_dir (str): 元の画像が格納されているディレクトリ
    output_labels_dir (str): ラベルファイルの出力ディレクトリ
    output_images_dir (str): 画像のコピー先ディレクトリ
    num_points_eyelid (int): 瞼の点の数
    num_points_iris (int): 虹彩の点の数
    num_points_pupil (int): 瞳孔の点の数
    start_folder (int): 処理を開始するフォルダ番号
    """
    # フォルダ名を数値としてソートするための関数
    def folder_num(name):
        try:
            return int(name)
        except ValueError:
            return float('inf')  # 数値でないフォルダ名は最後にソートされます

    # フォルダを数値順にソート
    folders = sorted([f for f in os.listdir(masks_dir) if os.path.isdir(os.path.join(masks_dir, f))], key=folder_num)

    # 開始フォルダ以降のフォルダのみを処理
    start_index = folders.index(str(start_folder)) if str(start_folder) in folders else 0
    folders = folders[start_index:]

    for folder_name in folders:
        folder_path = os.path.join(masks_dir, folder_name)
        png_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith(".png")]
        for file_name in tqdm(png_files, desc=f"Processing folder {folder_name}"):
            mask_file = os.path.join(folder_path, file_name)
            process_image(mask_file, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil)

if __name__ == '__main__':
    masks_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Masks"
    images_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/Images"
    output_labels_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/labels"
    output_images_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/images"

    # Create output directories if they don't exist
    os.makedirs(output_labels_dir, exist_ok=True)
    os.makedirs(output_images_dir, exist_ok=True)

    num_points_eyelid = 24
    num_points_iris = 16
    num_points_pupil = 12

    # 開始フォルダを指定（例：1番目のフォルダから開始）
    start_folder = 2

    process_all_images_in_directory(masks_dir, images_dir, output_labels_dir, output_images_dir, num_points_eyelid, num_points_iris, num_points_pupil, start_folder)

### 形式を調整

In [ ]:
### 枚数の確認

import os

def count_files(directory, include_bad=False):
    total_count = 0
    bad_count = 0
    for name in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, name)):
            total_count += 1
            if 'bad' in name.lower():
                bad_count += 1
    return total_count, bad_count

# ディレクトリパス
labels_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/labels/"
images_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/images/"

# ファイル数をカウント
labels_total, labels_bad = count_files(labels_dir)
images_total, images_bad = count_files(images_dir)

print(f"ラベルファイル総数: {labels_total}")
print(f"'bad'を含むラベルファイル数: {labels_bad}")
print(f"画像ファイル総数: {images_total}")
print(f"'bad'を含む画像ファイル数: {images_bad}")

if labels_total == images_total:
    print("ラベルと画像の総数が一致しています。")
else:
    print("警告: ラベルと画像の総数が一致していません。")

if labels_bad == images_bad:
    print("'bad'を含むラベルと画像の数が一致しています。")
else:
    print("警告: 'bad'を含むラベルと画像の数が一致していません。")

ラベルファイル総数: 3559
'bad'を含むラベルファイル数: 17
画像ファイル総数: 3559
'bad'を含む画像ファイル数: 17
ラベルと画像の総数が一致しています。
'bad'を含むラベルと画像の数が一致しています。


In [ ]:
"""
このスクリプトは以下の処理を行います：

1. 指定されたソースディレクトリ（画像とラベル）からファイルを読み込みます。
2. ファイル名に'bad'が含まれていないファイルをフィルタリングします。
3. フィルタリングされたファイルを訓練用と検証用に分割します（デフォルトで80:20の比率）。
4. 以下の新しい構造でディレクトリを作成し、ファイルをコピーします：
   /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/data/
   ├── images/
   │   ├── train/
   │   └── val/
   └── labels/
       ├── train/
       └── val/
5. tqdmを使用して、ファイルコピーの進行状況を表示します。
6. 処理されたファイル数を報告し、画像とラベルの数が一致しているか確認します。

注意：
- このスクリプトは既存のファイルを上書きします。実行前に出力ディレクトリの内容を確認してください。
- 大量のファイルを扱う場合でも、tqdmにより進行状況を確認できます。
"""

import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def copy_files(src_dir, dest_dir, files):
    os.makedirs(dest_dir, exist_ok=True)
    for file in tqdm(files, desc=f"Copying to {os.path.basename(dest_dir)}", unit="file"):
        shutil.copy2(os.path.join(src_dir, file), dest_dir)

def process_directory(src_dir, train_dir, val_dir, val_split=0.2):
    print(f"処理中のディレクトリ: {src_dir}")
    files = [f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f)) and 'bad' not in f.lower()]

    train_files, val_files = train_test_split(files, test_size=val_split, random_state=42)

    copy_files(src_dir, train_dir, train_files)
    copy_files(src_dir, val_dir, val_files)

    return len(train_files), len(val_files)

# ベースディレクトリ
base_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training"

# ソースディレクトリ
src_images_dir = os.path.join(base_dir, "images")
src_labels_dir = os.path.join(base_dir, "labels")

# 新しい出力ディレクトリ構造
output_dir = os.path.join(base_dir, "data")
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")

# 新しいディレクトリ構造の作成
train_images_dir = os.path.join(images_dir, "train")
val_images_dir = os.path.join(images_dir, "val")
train_labels_dir = os.path.join(labels_dir, "train")
val_labels_dir = os.path.join(labels_dir, "val")

# ファイルの処理とコピー
print("処理を開始します...")
train_images, val_images = process_directory(src_images_dir, train_images_dir, val_images_dir)
train_labels, val_labels = process_directory(src_labels_dir, train_labels_dir, val_labels_dir)

print(f"\n処理完了:")
print(f"訓練用画像: {train_images}")
print(f"検証用画像: {val_images}")
print(f"訓練用ラベル: {train_labels}")
print(f"検証用ラベル: {val_labels}")

if train_images == train_labels and val_images == val_labels:
    print("画像とラベルの数が一致しています。")
else:
    print("警告: 画像とラベルの数が一致していません。")

print(f"\n新しいデータセットの場所: {output_dir}")
print("新しいディレクトリ構造:")
print(f"{output_dir}/")
print("├── images/")
print("│   ├── train/")
print("│   └── val/")
print("└── labels/")
print("    ├── train/")
print("    └── val/")

処理を開始します...
Processing directory: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/images


Copying to images: 100%|██████████| 709/709 [00:18<00:00, 39.15file/s]


Processing directory: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/labels


Copying to labels: 100%|██████████| 709/709 [00:15<00:00, 45.35file/s]


処理完了:
訓練用画像: 2833
検証用画像: 709
訓練用ラベル: 2833
検証用ラベル: 709
画像とラベルの数が一致しています。

新しいデータセットの場所: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/filtered_data


In [ ]:
"""
このスクリプトは以下の処理を行います：

1. 指定されたベースディレクトリ内の 'images' と 'labels' フォルダを削除します。
2. 削除前にユーザーに確認を求めます。
3. 削除処理の結果を報告します。

注意：
- このスクリプトは元のデータを完全に削除します。実行前に十分注意し、必要なデータのバックアップを確保してください。
- 削除操作は取り消せません。確認プロンプトには慎重に回答してください。
"""

import os
import shutil

def remove_directory(dir_path):
    try:
        shutil.rmtree(dir_path)
        print(f"削除完了: {dir_path}")
    except Exception as e:
        print(f"エラー: {dir_path}の削除中に問題が発生しました。\n{str(e)}")

# ベースディレクトリ
base_dir = "/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training"

# ソースディレクトリ
src_images_dir = os.path.join(base_dir, "images")
src_labels_dir = os.path.join(base_dir, "labels")

print(f"削除対象のディレクトリ:")
print(f"1. {src_images_dir}")
print(f"2. {src_labels_dir}")

# 削除の確認
confirm = input("\n上記のフォルダを削除しますか？ この操作は取り消せません。(y/n): ")

if confirm.lower() == 'y':
    remove_directory(src_images_dir)
    remove_directory(src_labels_dir)
    print("\n指定されたフォルダの削除が完了しました。")
else:
    print("\n削除操作がキャンセルされました。フォルダは削除されていません。")

削除対象のディレクトリ:
1. /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/images
2. /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/labels

上記のフォルダを削除しますか？ この操作は取り消せません。(y/n): y
削除完了: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/images
削除完了: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/labels

指定されたフォルダの削除が完了しました。


In [ ]:
# YAMLファイルの作成

%%writefile /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/data  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/val # test images (optional)

# Classes
names:
  0: eyelid
  1: iris
  2: pupil

# Download script/URL (optional)
download: https://ultralytics.com/assets/coco128-seg.zip

Writing /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml


#**YOLOv8 training**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
#yolov8

/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training
fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics


In [4]:
from ultralytics import YOLO
#本体
#nc = 3に変更
# Load a model
model = YOLO('yolov8n-seg.yaml').load('yolov8n-seg.pt')  # build from YAML and transfer weights

Transferred 417/417 items from pretrained weights


In [ ]:
result =  model.train(data='/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml', epochs=30, imgsz=640)

Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.yaml, data=/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=yolov8n-seg.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

100%|██████████| 755k/755k [00:00<00:00, 22.2MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 58.1MB/s]


AMP: checks passed ✅


...

                   all        709       2127      0.956      0.921      0.955      0.743      0.946      0.909      0.947      0.649



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      16/30      2.97G     0.8545      1.618     0.4912      1.105          3        640: 100%|██████████| 178/178 [02:14<00:00,  1.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:20<00:00,  1.10it/s]


                   all        709       2127      0.964      0.921      0.956      0.752      0.952      0.908      0.948      0.654

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      17/30      2.98G     0.8856      1.617      0.489      1.117        113        640:  13%|█▎        | 23/178 [00:21<02:33,  1.01it/s]

In [5]:
## 続きからトレーニングする場合
from ultralytics import YOLO
#本体
#nc = 3に変更
# Load a model
model = YOLO('yolov8n-seg.yaml').load('yolov8n-seg.pt')  # build from YAML and transfer weights

result = model.train(data='/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml',
                     epochs=500,
                     imgsz=640,
                     resume='/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train2/weights/last.pt')  # チェックポイントファイルのパスを指定

Transferred 417/417 items from pretrained weights
Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.yaml, data=/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/eye_segmentation.yaml, epochs=500, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=yolov8n-seg.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=None, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_ma

100%|██████████| 755k/755k [00:00<00:00, 21.4MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/data/labels/train.cache... 2833 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2833/2833 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    441/500      2.97G     0.5116     0.9514     0.2616     0.9158          6        640: 100%|██████████| 178/178 [02:16<00:00,  1.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:21<00:00,  1.07it/s]


                   all        709       2127      0.974      0.966      0.976      0.798      0.966      0.959      0.974      0.712
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 341, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

441 epochs completed in 19.572 hours.
Optimizer stripped from /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train3/weights/last.pt, 6.8MB
Optimizer stripped from /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train3/weights/best.pt, 6.8MB

Validating /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train3/weights/best.pt...
Ultralytics YOLOv8.2.93 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:24<00:00,  1.05s/it]


                   all        709       2127      0.968      0.967      0.976      0.801      0.962      0.961      0.974      0.713
                eyelid        709        709      0.999          1      0.995      0.895      0.994      0.994      0.995      0.801
                  iris        709        709      0.998      0.999      0.995      0.908       0.98       0.98      0.988      0.747
                 pupil        709        709      0.907      0.903      0.937      0.599      0.913      0.908      0.938      0.592
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to /content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train3


### inference trained model

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# モデルの読み込み
model = YOLO("/content/drive/MyDrive/Deep_learning/Eyelid_segmentation/MOBIUS/YOLOv8_training/ultralytics/runs/segment/train3_458epochs/weights/best.pt")

# 画像のパスを指定
img_path = "/content/drive/MyDrive/Deep_learning/YOLACT_Mobius_ocular_dataset/testset(patient)/正常_小児_1.jpg"

# 元の画像を読み込む（BGRのまま）
original_img = cv2.imread(img_path)

# 推論の実行（元の画像をそのまま渡す）
results = model(original_img, save=True, save_txt=True)

# 結果の画像を取得
for r in results:
    result_img = r.plot()  # この結果はRGB形式

# 画像のリサイズ関数
def resize_image(image, width=None, height=None):
    if width is None and height is None:
        return image
    h, w = image.shape[:2]
    if width is None:
        aspect_ratio = height / h
        new_width = int(w * aspect_ratio)
        new_size = (new_width, height)
    else:
        aspect_ratio = width / w
        new_height = int(h * aspect_ratio)
        new_size = (width, new_height)
    return cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

# 画像のリサイズ
target_height = 500  # 表示する画像の高さを指定
original_img_resized = resize_image(original_img, height=target_height)
result_img_resized = resize_image(result_img, height=target_height)

# 2つの画像を横に連結（BGR形式のまま）
combined_img = np.hstack((original_img_resized, result_img_resized))

# 結果の表示
cv2_imshow(combined_img)

# 保存されたファイルの場所を表示（オプション）
print("保存された結果は以下のディレクトリにあります:")
print(model.predictor.save_dir)

## [試作品段階のコード](https://colab.research.google.com/drive/16fyns7E-PEstRELygVeMyLE5gbmVhyPe#scrollTo=NYNAK-r12mPz)

念のために共有します。